授業の内容を元に、ナイーブベイズを用いたスパムメール判定をするプログラムを組んでみよう  
つまり、関数wordProbと関数scoreを完成させること  
但し、sklearnを用いることは禁じる  
使ってもよいのはnumpy, MeCab, math, sys

## 形態素解析エンジンMeCabの導入

In [ ]:
!apt install aptitude
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3==0.7

- neologd辞書のインストール(すごく時間がかかる)

In [ ]:
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n

##文章から単語を取り出す関数parsewithelimination(sentense)

In [19]:
import MeCab
import numpy as np

def parsewithelimination(sentense):
  m=MeCab.Tagger()
  m.parse('')
  node=m.parseToNode(sentense)
    
  result=np.array([])
  while node:
    if node.feature.split(',')[6] == '*': # 原形を取り出す
      term=node.surface
    else :
      term=node.feature.split(',')[6]
        
    if node.feature.split(',')[1] in ['数','非自立','接尾']:
      node=node.next
      continue

    if node.feature.split(',')[0] in ['名詞', '動詞', '形容詞']:
      result=np.append(result, term)
        
    node=node.next

  return result

In [20]:
parsewithelimination('本日13時よりミーティングよろしくお願いいたします')

array(['本日', 'ミーティング', 'よろしい', 'お願い'], dtype='<U32')

## 訓練データの用意

In [21]:
import numpy as np

x=np.array(['今だけの激安謝恩セール、5割引', 
            '在庫処分特別セール開催中',
            '確認しないと損をする特別セール実施中',
            '本日のスケジュールの確認のお願いです',
            '進捗はいかがでしょうか。ご確認お願いいたします',
            '本日13時よりミーティングよろしくお願いいたします',
            '議事録を必ずご確認ください',
])

# 1をスパムメール、0を通常メールとする
y=np.array([1,1,1,0,0,0,0])

# 訓練データのスパムメール/普通メール内での出現回数を求める

In [22]:
categories=set()
wordset=set()
word_count={} 
category_count={}

for sent,cat in zip(x,y):
  words=parsewithelimination(sent)
  for w in words:
    if not cat in word_count:
      word_count[cat]={}
    if not w in word_count[cat]:
      word_count[cat][w]=0
    word_count[cat][w]+=1
    wordset.add(w)
  if not cat in category_count:
    category_count[cat]=0
  category_count[cat]+=1
  categories.add(cat)

In [23]:
categories

{0, 1}

In [24]:
wordset

{'いかが',
 'お願い',
 'する',
 'よろしい',
 'スケジュール',
 'セール',
 'ミーティング',
 '今',
 '処分',
 '割引',
 '在庫',
 '実施',
 '損',
 '本日',
 '激',
 '特別',
 '確認',
 '謝恩',
 '議事',
 '進捗',
 '開催'}

In [25]:
word_count[1]['セール']

3

In [26]:
word_count[0]['確認']

3

# 単語の条件付き確率$P(w_i|S)$ or $P(w_i|H)$を求める関数

In [27]:
def wordProb(word, category):
  a=0
  for i in word_count[category].values():
    a+=(i/len(word_count[category]))

  try:
    value=(word_count[category][word]+1)/(a+len(wordset))

  except KeyError:
    value=1/(a+len(wordset))

  return value

## 事後確率$log P(S|M)$ or $log P(S|H)$を求めるを求める関数

In [28]:
import math
def score(word_list, category):
  P=math.log(np.count_nonzero(y==category)/len(y))
  score_list=np.array([])

  for item in word_list:
    value=math.log(wordProb(item,category))
    score_list=np.append(score_list,value)

  score_list=np.append(score_list,P)
  score=score_list.sum()

  return score

## テキストを入力してスパムか普通かを判断

In [29]:
import sys
def predict(txt):
  best=None
  max=-sys.maxsize
  word_list=parsewithelimination(txt)
  score_list=[]
  for category in category_count.keys():
    value=score(word_list,category)
    score_list.append([category, value])
    if value > max:
      max=value
      best=category
  return best, score_list

In [30]:
predict('在庫割引処分実施中')

(1, [[1, -10.494435400223354], [0, -13.023541394757695]])

In [31]:
predict('明日ミーティングよろしくお願いいたします')

(0, [[1, -13.267024122463136], [0, -10.250952672517913]])